In [1]:
%load_ext memory_profiler

## Load Dependancies 

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.ensemble import RandomForestClassifier,RandomForestRegressor
from sklearn.linear_model import SGDRegressor
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
from sklearn.neighbors import KNeighborsClassifier,KNeighborsRegressor
from sklearn.svm import SVC,SVR

from sklearn import datasets
import scipy.stats as stats
import lightgbm as lgb
import xgboost as xgb

from sklearn.svm import SVC
from sklearn.datasets import load_digits
from sklearn.feature_selection import RFE
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from pathlib import Path
import pandas as pd
import numpy as np

import dask
from dask.distributed import worker_client
from dask.distributed import Client,LocalCluster


import joblib
import numpy as np
from dask.distributed import Client

import joblib
from sklearn.datasets import load_digits
from sklearn.model_selection import RandomizedSearchCV
from sklearn.svm import SVC

In [3]:
client = Client(scheduler_file='scheduler.json')
client

Client Scheduler: tcp://10.6.21.67:8787 Dashboard: http://10.6.21.67:8791/status,Cluster Workers: 96 Cores: 96 Memory: 404.96 GB


In [4]:
dask.config.set(scheduler='threads')


## Load Data

In [5]:
root = Path('/scratch/ge3/sg4953') 

In [6]:
# read the csv file created
train = np.genfromtxt(str(root)+'/input_dataset.csv', delimiter=',')
train = train.astype(np.float32)
train = train[~np.isnan(train).any(axis=1)]
train = train[~(train==-9999.0).any(axis=1)]

X, y = train[:, 1:], train[:, :1]
y = y.ravel()


In [7]:
# Set training and validation sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [10]:
# check for nodata values
# for i in range (train.shape[0]):
#     for j in range(train.shape[1]):
#         if train[i][j] == -9999.0:
#             print(i,j,train[i][j])
        

In [9]:
%%time
%%memit
#XGBoost

params={
    "max_depth":5, 
    "min_child_weight":2, 
    "eta": 1, 
    "colsample_bytree":0.8,  
    "eval_metric": "auc",
    'objective':'reg:squarederror'
}

clf = xgb.XGBModel(**params)


peak memory: 252.43 MiB, increment: 0.02 MiB
CPU times: user 88.6 ms, sys: 43.1 ms, total: 132 ms
Wall time: 281 ms


In [ ]:
%%time
%%memit
# Single Node

scores = cross_val_score(clf, X, y, cv=5,scoring='neg_mean_squared_error', n_jobs=-1)
print("MSE:"+ str(-scores.mean()))

In [10]:
%%time
%%memit
# Multiple Nodes 
with joblib.parallel_backend('dask'):
    scores = cross_val_score(clf, X, y, cv=5,scoring='neg_mean_squared_error')
print("MSE:"+ str(-scores.mean()))

MSE:0.1027763407677412
peak memory: 368.35 MiB, increment: 115.90 MiB
CPU times: user 231 ms, sys: 150 ms, total: 381 ms
Wall time: 1min 16s


In [ ]:
# Distributed Grid Search

In [11]:
from sklearn.model_selection import GridSearchCV

# Set the parameters by cross-validation
param_grid = {
        'min_child_weight': [1, 5, 10],
        'gamma': [0.5, 1, 1.5, 2, 5],
        'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.6, 0.8, 1.0],
        'max_depth': [3, 4, 5]
        }

xgb_model = xgb.XGBModel(
    learning_rate=0.02, 
    n_estimators=10, 
    objective="reg:squarederror"
)

grid_search = GridSearchCV(
    xgb_model,
    param_grid=param_grid,
    return_train_score=False,
    cv=3,
    scoring="precision",
    n_jobs=-1
)


In [14]:
# with joblib.parallel_backend('dask'):
grid_search.fit(X, y)

Task was destroyed but it is pending!
task: <Task pending coro=<DaskDistributedBackend._collect() running at /home/547/sg4953/.local/lib/python3.7/site-packages/joblib/_dask.py:198> wait_for=<Future pending cb=[<TaskWakeupMethWrapper object at 0x147f9133ad10>()]> cb=[IOLoop.add_future.<locals>.<lambda>() at /apps/pangeo/2020.05/lib/python3.7/site-packages/tornado/ioloop.py:690]>
/home/547/sg4953/.local/lib/python3.7/site-packages/sklearn/model_selection/_search.py:921: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan

GridSearchCV(cv=3,
             estimator=XGBModel(learning_rate=0.02, n_estimators=10,
                                objective='reg:squarederror'),
             n_jobs=-1,
             param_grid={'colsample_bytree': [0.6, 0.8, 1.0],
                         'gamma': [0.5, 1, 1.5, 2, 5], 'max_depth': [3, 4, 5],
                         'min_child_weight': [1, 5, 10],
                         'subsample': [0.6, 0.8, 1.0]},
             scoring='precision')

In [13]:
pd.DataFrame(grid_search.cv_results_).head()

AttributeError: 'GridSearchCV' object has no attribute 'cv_results_'

In [ ]:
from skopt import BayesSearchCV
from sklearn.svm import SVC

param_grid = {
        'min_child_weight': [1, 5, 10],
        'gamma': [0.5, 1, 1.5, 2, 5],
        'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.6, 0.8, 1.0],
        'max_depth': [3, 4, 5]
        }

xgb_model = xgb.XGBModel(
    learning_rate=0.02, 
    n_estimators=50, 
    objective=reg:squarederror
)


bayessearchcv = BayesSearchCV(
    xgb_model,
    search_spaces=param_grid,
    n_iter=10,
    cv=3,
    n_jobs=-1
)

# callback handler
def on_step(optim_result):
    score = searchcv.best_score_
    print("best score: %s" % score)
    if score >= 0.98:
        print('Interrupting!')
        return True


bayessearchcv.fit(X, y, callback=on_step)

In [9]:
client.shutdown()

distributed.client - ERROR - Failed to reconnect to scheduler after 10.00 seconds, closing client
ERROR:asyncio:_GatheringFuture exception was never retrieved
future: <_GatheringFuture finished exception=CancelledError()>
asyncio.exceptions.CancelledError


In [ ]:
cluster = LocalCluster(
    n_workers=2,
    processes=True,
    threads_per_worker=1
    )
    with Client(cluster) as client:
        x = np.linspace(0, 1, 100)
        y = x * x
        plt.plot(x, y)
        print('Computation complete! Stopping workers...')

    print('Execution complete!')


In [1]:
import dask
import xgboost as xgb
import pandas as pd
import numpy as np
import dask.array as da
import distributed
from dask.distributed import Client, wait, LocalCluster
from dask_cuda import LocalCUDACluster

# Start cluster and client. This is currently local, although I would like to make this distributed across many nodes.
# cluster = LocalCUDACluster(CUDA_VISIBLE_DEVICES='0,1,2,3', threads_per_worker=2)
cluster = LocalCluster()
client = Client(cluster)
client

Client Scheduler: tcp://127.0.0.1:37487 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 8 Cores: 48 Memory: 202.48 GB


In [2]:
import dask.dataframe as dd
df = dd.read_csv('input_dataset_filtered.csv')

In [3]:
X = df.drop("target",axis=1)   # Feature Matrix
y = df["target"]               # Target Variable

In [8]:
%%time
# Read in training data.
from dask_ml.model_selection import train_test_split
from sklearn.metrics import  explained_variance_score, mean_absolute_error, mean_squared_error, r2_score

scoring_function = [explained_variance_score, mean_absolute_error, mean_squared_error, r2_score][3]


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, train_size=0.75, random_state=0,shuffle=False)
# del X
# del y

# Create DaskDMatrix from X, y inputs.
dtrain = xgb.dask.DaskDMatrix(client=client, data=X_train, label=y_train)
dtest = xgb.dask.DaskDMatrix(client=client, data=X_test, label=y_test)

# # Delete X and y to free up memory.
# del X_train
# del X_eval
# del y_train
# del y_eval

# Train toy booster on 10 rounds with subsampling and gradient based sampling to reduce memory requirements.
bst = xgb.dask.train(
    client,
    {
#         'predictor': 'gpu_predictor', 
#         'tree_method': 'gpu_hist', 
#         'sampling_method': 'gradient_based',
        'verbosity': 2, 
        'objective': 'reg:squarederror',
        "evals_result":"evaluation_result",
        'subsample': 0.1
    }, 
    dtrain,
    num_boost_round=10
)

y_pred = xgb.dask.predict(client, bst, dtest).compute()
print(round(scoring_function(y_test, y_pred), 4))


0.6892
CPU times: user 636 ms, sys: 103 ms, total: 739 ms
Wall time: 6.1 s


In [4]:
from dask.distributed import Client, wait, LocalCluster
import distributed
client = distributed.client._get_global_client()
client

In [5]:
client.shutdown()


AttributeError: 'NoneType' object has no attribute 'shutdown'

In [15]:
import dask.dataframe as dd
df = dd.read_csv('input_dataset_filtered.csv')
df

,target,/g/data/ge3/sheece/LOC_distance_to_coast.tif,/g/data/ge3/sheece/mrvbf_9.tif,/g/data/ge3/sheece/relief_mrvbf_3s_mosaic.tif,/g/data/ge3/sheece/relief_elev_focalrange1000m_3s.tif,/g/data/ge3/sheece/relief_elev_focalrange300m_3s.tif,/g/data/ge3/sheece/saga_wetSM_85_resampled.tif,/g/data/ge3/sheece/tpi_300.tif,/g/data/ge3/sheece/slope_fill2.tif,/g/data/ge3/sheece/dem_fill.tif,/g/data/ge3/sheece/3dem_mag2.tif,/g/data/ge3/sheece/3dem_mag1_fin.tif,/g/data/ge3/sheece/3dem_mag0.fin.tif,/g/data/ge3/sheece/relief_roughness.tif,/g/data/ge3/sheece/LATITUDE_GRID1_clip.tif,/g/data/ge3/sheece/LATITUDE_GRID1_clip.tif.1,/g/data/ge3/sheece/Dose_2016.tif,/g/data/ge3/sheece/Potassium_2016.tif,/g/data/ge3/sheece/Thorium_2016.tif,/g/data/ge3/sheece/Rad2016U_Th.tif,/g/data/ge3/sheece/Rad2016K_Th.tif,/g/data/ge3/sheece/national_Wii_RF_multirandomforest_prediction.tif,/g/data/ge3/sheece/si_geol1.tif,/g/data/ge3/sheece/ceno_euc_aust1.tif,/g/data/ge3/sheece/Grav_lane_clip.tif,/g/data/ge3/sheece/be-30y-85m-avg-ND-NIR-GREEN.filled.lzw.nodata.tif,/g/data/ge3/sheece/be-30y-85m-avg-ND-RED-BLUE.filled.lzw.nodata.tif,/g/data/ge3/sheece/be-30y-85m-avg-ND-SWIR1-SWIR2.filled.lzw.nodata.tif,/g/data/ge3/sheece/be-30y-85m-avg_BLUE+SWIR2.tif,/g/data/ge3/sheece/be-30y-85m-avg-ND-SWIR1-NIR.filled.lzw.nodata.tif,/g/data/ge3/sheece/be-30y-85m-avg-CLAY-PC2.filled.lzw.nodata.tif,/g/data/ge3/sheece/be-30y-85m-avg-ND-RED-BLUE.filled.lzw.nodata.tif.1,/g/data/ge3/sheece/be-30y-85m-avg-RED.filled.lzw.nodata.tif,/g/data/ge3/sheece/be-30y-85m-avg-GREEN.filled.lzw.nodata.tif,/g/data/ge3/sheece/be-l8-all-85m-avg-BLUE.filled.lzw.nodata.tif,/g/data/ge3/sheece/be-l8-all-85m-avg-NIR.filled.lzw.nodata.tif,/g/data/ge3/sheece/be-30y-85m-avg-SWIR1.filled.lzw.nodata.tif,/g/data/ge3/sheece/be-30y-85m-avg-SWIR2.filled.lzw.nodata.tif,/g/data/ge3/sheece/s2-dpca-85m.tif,/g/data/ge3/sheece/water-85m.tif,/g/data/ge3/sheece/clim_EPA_albers.tif,/g/data/ge3/sheece/Clim_Prescott_LindaGregory.tif,/g/data/ge3/sheece/clim_PTA_albers.tif,/g/data/ge3/sheece/clim_WDA_albers.tif,/g/data/ge3/sheece/clim_RSM_albers.tif
npartitions=4,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [16]:
df = df.compute()

In [19]:
df.shape

(489317, 45)

In [73]:
test = df.drop(df.head(100000).index)
test.shape


(160571, 45)

In [74]:
489317-160571

328746

In [33]:
df.head(5).index

Int64Index([0, 1, 2, 3, 4], dtype='int64')

In [34]:
df.head(4).index

Int64Index([0, 1, 2, 3], dtype='int64')

In [52]:
489317/395733

1.2364826789779977

## In-house Recursive feature elimination


In [10]:
from sklearn.linear_model import LogisticRegressionCV
from dask_ml.wrappers import ParallelPostFit
import dask.dataframe as dd
# from sklearn.model_selection import train_test_split
from dask_ml.model_selection import train_test_split
from sklearn.metrics import  explained_variance_score, mean_absolute_error, mean_squared_error, r2_score
from sklearn import preprocessing
from sklearn import utils

In [13]:
df = dd.read_csv('input_dataset_filtered.csv')
X = df.drop("target",axis=1).compute()   # Feature Matrix
y = df["target"].compute()               # Target Variable

lab_enc = preprocessing.LabelEncoder()
y = lab_enc.fit_transform(y)
del df
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0, shuffle=False)



In [14]:
clf = ParallelPostFit(LogisticRegressionCV(cv=3), scoring="r2")
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

/home/547/sg4953/venvs/jupyter/lib/python3.7/site-packages/sklearn/model_selection/_split.py:668: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  % (min_groups, self.n_splits)), UserWarning)


MemoryError: Unable to allocate 282. GiB for an array with shape (244658, 154656) and data type int64

In [ ]:
r2_score(y_test, y_pred)

In [ ]:
clf.score(X_test, y_test)